In [1]:
import cv2
import numpy as np

In [2]:
cam = cv2.VideoCapture("resources/couple.mp4")

In [3]:
# Check if camera opened successfully
if (cam.isOpened()== False): 
  print("Error opening video stream or file")

# Displaying image frames per second
fps = cam.get(cv2.CAP_PROP_FPS)
print(f"The frame rate is: {fps}")

# define the number of frames per second
new_fps = 10

# determine fps scale factor
scale_fps = int(fps / new_fps)
print(f"new frame rate scale is: {scale_fps}")

The frame rate is: 30.0
new frame rate scale is: 3


In [4]:
# Create a window to hold the video frame
cv2.namedWindow("video", cv2.WINDOW_NORMAL)

# Create a list to hold the previously cropped images
previouSampleFrames = {}

# The number of sample ROIs
sampleCount = 5

# tells us whether the first frame is the first
isFirstFrame = False


In [ ]:
frameCount = 0
while(cam.isOpened() == True):
    isRet, frame = cam.read()
    if(isRet == True & (frameCount%scale_fps == 0)):
        
        #crop out the sample Regions
        for i in range(sampleCount):
            newSampleFrame = frame[int((i / sampleCount) * (frame.shape[0] / 2)) : int(frame.shape[0] / 2 + (i / sampleCount) * (frame.shape[0] / 2)) , : ]
            cv2.imshow(f"Slice {i}", newSampleFrame)
            cv2.waitKey(5)
            
        # Initiate ORB detector
        orb = cv2.ORB_create()

        # find the keypoints with ORB
        kp = orb.detect(frame,None)

        # compute the descriptors with ORB
        kp, des = orb.compute(frame, kp)

        # draw only keypoints location,not size and orientation
        frame = cv2.drawKeypoints(frame, kp, None, color=(0,255,0), flags=0)
        
        cv2.imshow("video", frame)
        
    elif(isRet == False):
        break
    
    if (cv2.waitKey(20) & 0xFF == ord('q')):
        break;
    
# Increment the freme count
    frameCount += 1


In [ ]:
# release resources
cam.release()
cv2.destroyAllWindows()